In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Datasets/Twitter_Data.csv")

In [2]:
df = df[df["category"]!=0]
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [3]:
import re
def remove_special_words(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'[^\w\s\U0001F600-\U0001F64F]','',text) # Remove any special chars but emojis
    return text

df["clean_text"] = df["clean_text"].apply(remove_special_words)
df["clean_text"] = df["clean_text"].str.lower()

In [4]:
df.head()

,clean_text,category
0,when modi promised minimum government maximum ...,-1.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
8,with upcoming election india saga going import...,1.0


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
sequences = tokenizer.texts_to_sequences(df['clean_text'])
padded_sequences = pad_sequences(sequences, padding='post')

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences,
                                                    df['category'],
                                                    test_size=0.2,
                                                    random_state=42)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True)
checkpoint_path = '/content/drive/MyDrive/models/twitter_sentiment_analysis/best_model.h5'
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=0)
lr_reduce_callback = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.34,
    patience=2,
    min_delta=1e-4,
    cooldown=2,
    min_lr=1e-7,
    verbose=0)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

model = Sequential()
model.add(Embedding(
    input_dim=len(tokenizer.word_index) + 1,
    output_dim=128,
    input_length=padded_sequences.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [9]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0001)
model.compile(  loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'],)

In [ ]:
history = model.fit(X_train, y_train,
            epochs=30,
            batch_size=64,
            validation_data=(X_test, y_test),
            callbacks=[early_stopping,lr_reduce_callback,checkpoint_callback])

Epoch 1/30
1347/1347 [==============================] - ETA: 0s - loss: -0.2341 - accuracy: 0.2229

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1347/1347 [==============================] - 688s 506ms/step - loss: -0.2341 - accuracy: 0.2229 - val_loss: -3.2363 - val_accuracy: 0.5888 - lr: 1.0000e-04
Epoch 2/30
1347/1347 [==============================] - 678s 503ms/step - loss: -5.2843 - accuracy: 0.5537 - val_loss: -8.2217 - val_accuracy: 0.5583 - lr: 1.0000e-04
Epoch 3/30
1347/1347 [==============================] - 672s 499ms/step - loss: -9.3493 - accuracy: 0.5730 - val_loss: -10.6292 - val_accuracy: 0.6091 - lr: 1.0000e-04
Epoch 4/30
1347/1347 [==============================] - 655s 486ms/step - loss: -13.9704 - accuracy: 0.5659 - val_loss: -14.6261 - val_accuracy: 0.5671 - lr: 1.0000e-04
Epoch 5/30
1347/1347 [==============================] - 660s 490ms/step - loss: -18.5573 - accuracy: 0.5786 - val_loss: -19.9725 - val_accuracy: 0.5647 - lr: 1.0000e-04
Epoch 6/30
1347/1347 [==============================] - 651s 483ms/step - loss: -23.7451 - accuracy: 0.5996 - val_loss: -19.8305 - val_accuracy: 0.5736 - lr: 1.0000e-04
Ep

In [ ]:
pd.DataFrame(history.history).to_csv("/content/drive/MyDrive/models/twitter_sentiment_analysis/history.csv")